In [9]:
import os
import pandas as pd

# Define the paths to your dataset
dataset_paths = [
    r'C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\test',
    r'C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\train',
    r'C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\val'
]

# Initialize empty lists to store full paths of filenames and corresponding classes
full_paths = []
classes = []

# Iterate over each dataset path
for dataset_path in dataset_paths:
    # List all subdirectories in the dataset_path
    subdirectories = [os.path.join(dataset_path, d) for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
    
    # Loop through each subdirectory
    for subdir in subdirectories:
        # Get the class name from the last part of the path
        class_name = os.path.basename(subdir)
        
        # List all files in the subdirectory
        files = [os.path.join(subdir, f) for f in os.listdir(subdir) if os.path.isfile(os.path.join(subdir, f))]
        
        # Append full paths of filenames and their corresponding classes to the lists
        full_paths.extend(files)
        classes.extend([class_name] * len(files))

# Create a dataframe from the lists
df = pd.DataFrame({'filename': full_paths, 'class': classes})

# Get class counts
class_counts = df['class'].value_counts()

# Display the class counts
print(class_counts)


class
Melanocytic nevus       505
Melanoma                330
Basal cell carcinoma    248
Benign keratosis        168
Actinic keratosis       167
Name: count, dtype: int64


In [10]:
df

,filename,class
0,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\tes...,Actinic keratosis
1,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\tes...,Actinic keratosis
2,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\tes...,Actinic keratosis
3,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\tes...,Actinic keratosis
4,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\tes...,Actinic keratosis
...,...,...
1413,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\val...,Melanoma
1414,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\val...,Melanoma
1415,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\val...,Melanoma
1416,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\val...,Melanoma


In [12]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Step 1: Load the data from the provided table
data = df

# Step 2: Load images into arrays
def load_images(image_paths):
    images = []
    for path in image_paths:
        img = load_img(path, target_size=(100, 100))  # Resize the image to (150, 150)
        img = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return np.array(images)

# Load images and labels
image_paths = data['filename']
images = load_images(image_paths)
labels = data['class']

# Step 3: Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)


In [ ]:
# Step 4: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)

# Step 5: Create a CNN model
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output before feeding into the fully connected layers
model.add(Flatten())

# Add fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))

# Output layer with softmax activation for multi-class classification
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Step 6: Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(len(label_encoder.classes_), activation='softmax')(x)

# Combine the base model with custom classification head
model_vgg16 = Model(inputs=base_model.input, outputs=output)

# Compile the model
model_vgg16.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model_vgg16.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load pre-trained DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(len(label_encoder.classes_), activation='softmax')(x)

# Combine the base model with custom classification head
model_densenet = Model(inputs=base_model.input, outputs=output)

# Compile the model
model_densenet.compile(optimizer='adam',
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

# Train the model
model_densenet.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
